In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 5
pd.options.display.max_columns = 21

from nycflights13 import flights, weather, planes, airports, airlines

Es.1 Voli il giorno di Natale

In [2]:
voliNatale = flights.loc[(flights['month'] == 12) & (flights['day'] == 25)]
#voliNatale

Es.2 Voli con un ritardo in partenza o in arrivo

In [4]:
voliRitardo = flights.loc[(flights['dep_delay'] > 0) | (flights['arr_delay'] > 0) ].dropna()
#voliRitardo

Es.3 Voli con ritardo in partenza ma non in arrivo + PrimaryKey

In [7]:
flID = flights.copy()
primaryKey = np.arange(0, len(flights), 1)
flID.insert(0, "id",primaryKey)
flID.loc[(flID['dep_delay'] > 0) & (flID['arr_delay'] < 0) ]



,id,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
19,19,2013,1,1,601.0,600,1.0,844.0,850,-6.0,B6,343,N644JB,EWR,PBI,147.0,1023,6,0,2013-01-01T11:00:00Z
47,47,2013,1,1,644.0,636,8.0,931.0,940,-9.0,UA,1701,N75435,EWR,FLL,151.0,1065,6,36,2013-01-01T11:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336744,336744,2013,9,30,2121.0,2100,21.0,2349.0,14,-25.0,DL,2363,N193DN,JFK,LAX,296.0,2475,21,0,2013-10-01T01:00:00Z
336749,336749,2013,9,30,2129.0,2059,30.0,2230.0,2232,-2.0,EV,5048,N751EV,LGA,RIC,45.0,292,20,59,2013-10-01T00:00:00Z
